In [1]:
import numpy as np
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
from copy import copy
from datetime import datetime, timedelta

from stonesoup.functions import gm_reduce_single

from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.imm import IMMPredictor
from stonesoup.updater.imm import IMMUpdater
from stonesoup.models.transition.linear import ConstantVelocity, \
    CombinedLinearGaussianTransitionModel, LinearGaussianTimeInvariantTransitionModel, RandomWalk

from stonesoup.models.measurement.linear import LinearGaussian

from stonesoup.types.state import GaussianState
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.detection import Detection
from stonesoup.types.state import StateVector, CovarianceMatrix, \
    GaussianMixtureState, WeightedGaussianState
from stonesoup.types.track import Track

from stonesoup.simulator.simple import SingleTargetGroundTruthSimulator

from matplotlib.patches import Ellipse


In [2]:
from bayou.datastructures import Gaussian as EMGPB2Gaussian
from bayou.datastructures import GMM as EMGPB2GMM
from bayou.datastructures import GMMSequence as EMGPB2GMMSequence
from bayou.models import LinearModel as EMGPB2LinearModel
from bayou.models import ConstantVelocity as EMGPB2ConstantVelocity
from bayou.expmax.skf import SKF

Plot function

In [3]:
def plot_cov_ellipse(cov, pos, nstd=2, ax=None, **kwargs):
    """
    Plots an `nstd` sigma error ellipse based on the specified covariance
    matrix (`cov`). Additional keyword arguments are passed on to the
    ellipse patch artist.
    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """

    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:, order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta,
                    **kwargs)

    ax.add_artist(ellip)
    return ellip


Groundtruth and detections

In [4]:
gt_track = []
gt_time = []
iter_model_1 = 120
iter_model_2 = 80

gt_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(2.0 ** 2),
                        RandomWalk(2.0 ** 2))
                        )

timestamp_init = datetime.now()
state_init_1 = GaussianState(StateVector([[0], [0]]),
                           CovarianceMatrix(np.diag([0.0, 0.0])),
                           timestamp=timestamp_init)
gt_generator_1 = SingleTargetGroundTruthSimulator(gt_transition_model_1, state_init_1, number_steps=iter_model_1)

for time, gnd_paths in gt_generator_1.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

gt_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(10.0 ** 2),
                        RandomWalk(10.0 ** 2))
                        )
state_init_2 = GaussianState(gt_track[-1].state_vector,
                           CovarianceMatrix(np.diag([0.0, 0.0])),
                           timestamp=gt_time[-1])
gt_generator_2 = SingleTargetGroundTruthSimulator(gt_transition_model_2, state_init_2, number_steps=iter_model_2)
for time, gnd_paths in gt_generator_2.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

detection_track = []
mesurement_noise_1 = np.diag([0.5 ** 2, 0.5 ** 2])
measurement_model_1 = LinearGaussian(ndim_state=2, mapping=[0, 1], noise_covar=mesurement_noise_1)
for i in range(iter_model_1):
    m_ = Detection(measurement_model_1.function(gt_track[i].state_vector, measurement_model_1.rvs(1)), gt_time[i])
    detection_track.append(m_)

mesurement_noise_2 = np.diag([0.75 ** 2, 0.75 ** 2])
measurement_model_2 = LinearGaussian(ndim_state=2, mapping=[0, 1], noise_covar=mesurement_noise_2)
for i in range(iter_model_2):
    ii = i + iter_model_1
    m_ = Detection(measurement_model_2.function(gt_track[ii].state_vector, measurement_model_2.rvs(1)), gt_time[ii])
    detection_track.append(m_)

# print(gt_track)
# print(detection_track)
# print(gt_time)

Filter model. Testing without EMGPB2 estimated parameters.

In [5]:
ft_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(1.0 ** 2),
                        RandomWalk(1.0 ** 2))
                        )
ft_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(20.0 ** 2),
                        RandomWalk(20.0 ** 2))
                        )
ft_measurement_noise = (2.0 **2) * np.eye(2)

ft_measurement_model = LinearGaussian(ndim_state=2, mapping=[0, 1],
                                   noise_covar=ft_measurement_noise)

ft_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
ft_predictor_1 = KalmanPredictor(ft_transition_model_1)
ft_predictor_2 = KalmanPredictor(ft_transition_model_2)
ft_imm_predictor = IMMPredictor([ft_predictor_1, ft_predictor_2], ft_model_transition_matrix)

ft_updater = KalmanUpdater(ft_measurement_model)
ft_imm_updater = IMMUpdater([ft_updater, ft_updater], ft_model_transition_matrix)

ft_state_init = WeightedGaussianState(StateVector([[0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)):
    measurement = detection_track[i]
    # State prediction
    prediction = ft_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = ft_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = ft_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 1.8058316100421927


Filter model. Testing with correct parameters.

In [6]:
model_transition_matrix = np.array([[0.3, 0.7],
                                    [0.3, 0.7]])

predictor_1 = KalmanPredictor(gt_transition_model_1)
predictor_2 = KalmanPredictor(gt_transition_model_2)
imm_predictor = IMMPredictor([predictor_1, predictor_2],
                             model_transition_matrix)

updater_1 = KalmanUpdater(measurement_model_1)
updater_2 = KalmanUpdater(measurement_model_2)
imm_updater = IMMUpdater([updater_1, updater_2], model_transition_matrix)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)):
    measurement = detection_track[i]
    # State prediction
    prediction = imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print('----------------------')
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.9031139013004704


EMGPB2

In [7]:
F = np.asarray([
        [1, 0],
        [0, 1],
    ])
H = np.asanyarray([
    [1, 0],
    [0, 1]
])
def get_Q(Q_sig):
    Q = (Q_sig ** 2) * np.diag([1.0, 1.0])
    return Q
def transfer_Q(Q):
    new_Q = np.zeros_like(Q)
    for i in range(new_Q.shape[0]):
        new_Q[i, i] = Q[i, i]
    return new_Q

In [ ]:
g1 = EMGPB2Gaussian(np.zeros([2, 1]), 10.0*np.eye(2))
g2 = EMGPB2Gaussian(np.zeros([2, 1]), 10.0*np.eye(2))
initial_gmm_state = EMGPB2GMM([g1, g2])

# measurements = 5 * np.random.randn(200, 2, 1) + 1

detection_track_EMGPB2 = [ele.state_vector for ele in detection_track]
gmmsequence = EMGPB2GMMSequence(np.float64(detection_track_EMGPB2), initial_gmm_state)

m1 = EMGPB2LinearModel(F, get_Q(1.0), H, (2.0 ** 2)*np.eye(2))
m2 = EMGPB2LinearModel(F, get_Q(20.0), H, (2.0 ** 2)*np.eye(2))
initial_models = [m1, m2]

Z = np.array([[0.5, 0.5], 
              [0.5, 0.5]])
dataset = [gmmsequence]

new_models_all, Z, dataset, LL = SKF.EM(dataset, initial_models, Z,
                                    max_iters=100, threshold=1e-6, learn_H=False, learn_R=True,
                                    learn_A=False, learn_Q=True, learn_init_state=False, learn_Z=True,
                                    diagonal_Q=False, wishart_prior=False)
new_models = new_models_all[-1]

new_models[0].Q = (new_models[0].Q + new_models[0].Q.T)/2
new_models[1].Q = (new_models[1].Q + new_models[1].Q.T)/2

-1264.539955037816
model -- 0 new_Q: 
[[2.79655907 0.37208049]
 [0.40007881 3.17291919]]
model -- 1 new_Q: 
[[154.2666222  -20.8821184 ]
 [-20.83221527  75.15109065]]
model -- 0 new_R: 
[[ 3.48917114 -0.08430158]
 [-0.08430158  3.96296042]]
model -- 1 new_R: 
[[5.23870925 0.40393242]
 [0.40393242 5.94906254]]
new_Z: 
[[0.71290087 0.28207401]
 [0.71290087 0.28207401]]
-----------------------------------------------
-1203.5631658002892
model -- 0 new_Q: 
[[4.51773791 0.57421156]
 [0.5838037  5.0920974 ]]
model -- 1 new_Q: 
[[147.71611041 -24.38878456]
 [-24.3675221   63.72139934]]
model -- 0 new_R: 
[[ 3.12288827 -0.0112964 ]
 [-0.01175469  3.48362864]]
model -- 1 new_R: 
[[5.59331432 0.50020423]
 [0.49661879 6.60050394]]
new_Z: 
[[0.72239038 0.2725845 ]
 [0.72239038 0.2725845 ]]
-----------------------------------------------
-1205.2545408836656
model -- 0 new_Q: 
[[5.83219777 0.60173613]
 [0.60510184 6.5552147 ]]
model -- 1 new_Q: 
[[148.61504486 -26.00455611]
 [-26.00494945  60.265912

-1208.06364273279
model -- 0 new_Q: 
[[6.91781137 0.54978796]
 [0.549774   8.28795659]]
model -- 1 new_Q: 
[[150.88706399 -23.16193093]
 [-23.16485076  60.85685705]]
model -- 0 new_R: 
[[1.67287695 0.33945957]
 [0.33944578 1.99328566]]
model -- 1 new_R: 
[[2.64843246 0.79730522]
 [0.79728095 3.72191195]]
new_Z: 
[[0.72138782 0.27358706]
 [0.72138782 0.27358706]]
-----------------------------------------------
-1207.7567181727782
model -- 0 new_Q: 
[[6.85493021 0.54357812]
 [0.54355404 8.24454561]]
model -- 1 new_Q: 
[[150.41254398 -22.84705397]
 [-22.84957713  60.83726332]]
model -- 0 new_R: 
[[1.63636761 0.33581902]
 [0.33580783 1.95899987]]
model -- 1 new_R: 
[[2.56813797 0.77302622]
 [0.77300692 3.62646105]]
new_Z: 
[[0.71967986 0.27529502]
 [0.71967986 0.27529502]]
-----------------------------------------------
-1207.4673238667867
model -- 0 new_Q: 
[[6.7933965  0.53640778]
 [0.53637455 8.20517342]]
model -- 1 new_Q: 
[[149.96033157 -22.54351318]
 [-22.54571218  60.80804501]]
mode

-1204.075284930455
model -- 0 new_Q: 
[[5.91960806 0.36316682]
 [0.36317603 7.83878843]]
model -- 1 new_Q: 
[[144.13305232 -18.70278973]
 [-18.70418957  59.76442364]]
model -- 0 new_R: 
[[1.15771485 0.24954538]
 [0.24954578 1.51409194]]
model -- 1 new_R: 
[[1.58214723 0.4483309 ]
 [0.44833157 2.49597534]]
new_Z: 
[[0.69511312 0.29986175]
 [0.69511312 0.29986175]]
-----------------------------------------------
-1203.957313985407
model -- 0 new_Q: 
[[5.88571826 0.35434611]
 [0.35436726 7.82754427]]
model -- 1 new_Q: 
[[143.89639693 -18.56119699]
 [-18.5626262   59.71024634]]
model -- 0 new_R: 
[[1.14075813 0.24584858]
 [0.2458489  1.49803033]]
model -- 1 new_R: 
[[1.55044329 0.4383365 ]
 [0.43833677 2.45898065]]
new_Z: 
[[0.69416651 0.30080836]
 [0.69416651 0.30080836]]
-----------------------------------------------
-1203.8450268673275
model -- 0 new_Q: 
[[5.85339223 0.34575351]
 [0.345787   7.81679138]]
model -- 1 new_Q: 
[[143.66710674 -18.42534285]
 [-18.42679992  59.65807193]]
mode

-1202.5447077790736
model -- 0 new_Q: 
[[5.47987378 0.22761665]
 [0.22786747 7.68155701]]
model -- 1 new_Q: 
[[140.58188313 -16.70824462]
 [-16.70985345  59.04341676]]
model -- 0 new_R: 
[[0.91019887 0.19816638]
 [0.19816395 1.26645273]]
model -- 1 new_R: 
[[1.14880747 0.32081156]
 [0.32080309 1.94829928]]
new_Z: 
[[0.68105093 0.31392395]
 [0.68105093 0.31392395]]
-----------------------------------------------


Filter model. Testing with learnt parameters from EMGPB2.

In [ ]:
# em_transition_model_1 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[0].A, covariance_matrix=transfer_Q(new_models[0].Q))
# em_transition_model_2 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[1].A, covariance_matrix=transfer_Q(new_models[1].Q))

em_transition_model_1 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[0].A, covariance_matrix=new_models[0].Q)
em_transition_model_2 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[1].A, covariance_matrix=new_models[1].Q)
em_measurement_noise_1 = new_models[0].R
em_measurement_noise_2 = new_models[1].R

em_measurement_model_1 = LinearGaussian(ndim_state=2, mapping=[0, 1],
                                   noise_covar=em_measurement_noise_1)
em_measurement_model_2 = LinearGaussian(ndim_state=2, mapping=[0, 1],
                                   noise_covar=em_measurement_noise_2)

em_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
em_predictor_1 = KalmanPredictor(em_transition_model_1)
em_predictor_2 = KalmanPredictor(em_transition_model_2)
em_imm_predictor = IMMPredictor([em_predictor_1, em_predictor_2], em_model_transition_matrix)

em_updater_1 = KalmanUpdater(em_measurement_model_1)
em_updater_2 = KalmanUpdater(em_measurement_model_2)
em_imm_updater = IMMUpdater([em_updater_1, em_updater_2], em_model_transition_matrix)

em_state_init = WeightedGaussianState(StateVector([[0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(em_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)):
    measurement = detection_track[i]
    
    # State prediction
    prediction = em_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = em_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = em_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(em_measurement_model_1.function(track[i].state_vector, np.zeros((2,1))) - em_measurement_model_1.function(gt_track[i].state_vector, np.zeros((2,1))))
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

In [ ]:
len(rmse_tmp)